# Fussing for Clancy

In [25]:
# imports
import pandas
from importlib import resources
import os
from frb.frb import FRB

from astropath.priors import load_std_priors
from astropath.path  import PATH

# Load

In [15]:
my_frb = FRB.by_name('FRB20181112A')

In [16]:
ppath = os.path.join(resources.files('frb'), 'data', 'Galaxies', 'PATH')
pfile = os.path.join(ppath, f'{my_frb.frb_name}_PATH.csv')
ptbl = pandas.read_csv(pfile)

In [17]:
ptbl.head()

,Unnamed: 0,label,xcentroid,ycentroid,sky_centroid,bbox_xmin,bbox_xmax,bbox_ymin,bbox_ymax,area,...,separation,ang_size,P_c,Sigma_m,mag,P_O,p_xO,P_Ox,P_Ux,P_U
0,7,8,57.785190,59.984241,NaN,49,66,52,67,199.0,...,0.400133,0.813759,0.062768,0.000574,21.474892,0.085197,1.403596e-02,0.616352,0.0,0.0
1,8,9,49.561525,78.883265,NaN,37,64,64,97,643.0,...,5.468470,1.208805,0.012283,0.000057,19.175204,0.856187,8.655226e-04,0.381955,0.0,0.0
2,1,2,60.193557,31.721416,NaN,54,67,24,38,138.0,...,6.957142,0.719079,0.227394,0.000982,22.058893,0.049780,6.589744e-05,0.001691,0.0,0.0
3,2,3,74.992777,35.031912,NaN,72,77,32,39,36.0,...,7.285763,0.429485,0.779461,0.005533,24.101438,0.008836,6.712320e-07,0.000003,0.0,0.0


# Priors

In [18]:
# Load prior
priors = load_std_priors()
prior = priors['adopted'] # Default

In [19]:
theta_new = dict(method='exp', 
    max=priors['adopted']['theta']['max'], 
    scale=0.5)
prior['theta'] = theta_new

# Candidates

In [22]:
candidates = ptbl[['ang_size', 'mag', 'ra', 'dec', 'separation']]

In [23]:
candidates.head()

,ang_size,mag,ra,dec,separation
0,0.813759,21.474892,327.348630,-52.970904,0.400133
1,1.208805,19.175204,327.349581,-52.969584,5.468470
2,0.719079,22.058893,327.348354,-52.972876,6.957142
3,0.429485,24.101438,327.346639,-52.972646,7.285763


In [30]:
candidates.mag.values

array([21.47489241, 19.17520415, 22.05889302, 24.10143842])

# PATH time

In [34]:
path_181112 = PATH()
path_181112.init_candidates(candidates.ra.values,
                         candidates.dec.values,
                         candidates.ang_size.values,
                         mag=candidates.mag.values)
path_181112.frb = my_frb

## Localization

In [39]:
frb_eellipse = dict(a=my_frb.sig_a,
                    b=my_frb.sig_b,
                    theta=my_frb.eellipse['theta'])

In [40]:
path_181112.init_localization('eellipse', 
                            center_coord=path_181112.frb.coord,
                            eellipse=frb_eellipse)

## Priors

In [41]:
path_181112.init_cand_prior('inverse', P_U=prior['U'])
path_181112.init_theta_prior(prior['theta']['method'], 
                            prior['theta']['max'],
                            prior['theta']['scale'])

In [42]:
path_181112.calc_priors()                            

array([0.0851965 , 0.85618737, 0.04978019, 0.00883594])

## Posterior

In [45]:
# Calculate p(O_i|x)
debug = True
path_181112.calc_posteriors('fixed', 
                         box_hwidth=10., 
                         max_radius=10., 
                         debug=debug)

(array([9.06433652e-01, 9.32671270e-02, 2.98469299e-04, 7.51464714e-07]),
 np.float64(0.0))

In [47]:
path_181112.candidates.sort_values('mag')

,ra,dec,ang_size,mag,P_O,p_xO,P_Ox,P_Ux
1,327.349581,-52.969584,1.208805,19.175204,0.856187,1.762590e-04,9.326713e-02,0.0
0,327.348630,-52.970904,0.813759,21.474892,0.085197,1.721495e-02,9.064337e-01,0.0
2,327.348354,-52.972876,0.719079,22.058893,0.049780,9.701406e-06,2.984693e-04,0.0
3,327.346639,-52.972646,0.429485,24.101438,0.008836,1.376092e-07,7.514647e-07,0.0
